In [2]:
# Imports
import numpy as np
import pandas as pd
import nltk
import contractions
import math
import re
import string
from google.cloud import bigquery

In [3]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

client = bigquery.Client(location="US", project="helio-staging")


Client creating using default project: helio-staging


In [4]:
parent_cat = ["food",
                    "alcoholic beverages",
                    "personal care & beauty",
                    "Alcoholic Beverages",
                    "Pet Products",
                    "pet products",
                    "Household Consumables",
                    "household consumables",
                    "Personal Care & Beauty",
                    "Food",
                    "non-alcoholic beverages"]

In [7]:
job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ArrayQueryParameter("parent_cat", "STRING", parent_cat)
        ]
)
max_date_query = """
    Select T1.normalized_url, T1.brand_attribute, T1.primary_category, T2.p12m_monthly_dollars, T2.p12m_monthly_dollars_yoy_pct_growth
    From (SELECT normalized_url, brand_attribute, parent_category, primary_category FROM `helio-staging.attributes.brand_attributes_cleaned__1_0` 
    Where parent_category in ("food"," alcoholic beverages ","personal care & beauty","Personal Care & Beauty","Pet Products","Alcoholic Beverages","pet products","Food","non-alcoholic beverages","Non-Alcoholic Beverages") ) as T1
    Join (SELECT normalized_url, fct_date, p12m_monthly_dollars,p12m_monthly_dollars_yoy_pct_growth from `helio-staging.nielsen.nielsen__monthly_offline_sales__2_0` WHERE execution_date = '2022-05-05' AND fct_date = '2021-12') as T2
    on T1.normalized_url = T2.normalized_url
    Where p12m_monthly_dollars >= 1000000 AND p12m_monthly_dollars<=20000000
    ORDER BY T2.p12m_monthly_dollars_yoy_pct_growth DESC
    """
query_job = client.query(
    max_date_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config
)  # API request - starts the query

attributes = query_job.to_dataframe()
attributes = attributes.loc[reviews.astype(str).drop_duplicates().index]

NameError: name 'reviews' is not defined

In [8]:
attributes

,normalized_url,brand_attribute,primary_category,p12m_monthly_dollars,p12m_monthly_dollars_yoy_pct_growth
0,ignite.co,bean,Vitamins & Supplements,6.228188e+06,5276.378769
1,ignite.co,actionbutton,Vitamins & Supplements,6.228188e+06,5276.378769
2,ignite.co,green_coffee,Vitamins & Supplements,6.228188e+06,5276.378769
3,ignite.co,topicals_cbd,Vitamins & Supplements,6.228188e+06,5276.378769
4,ignite.co,series,Vitamins & Supplements,6.228188e+06,5276.378769
...,...,...,...,...,...
11829962,mixbar.com,fragrance_collection,Nail Care,1.392238e+07,NaN
11829963,fresh-innovations.com,organic_apple,Packaged & Prepared Food,1.559645e+06,NaN
11829964,diabetesdoctor.com,insulin,Vitamins & Supplements,1.752599e+06,NaN
11829965,txranchwaterco.com,texas_ranch,Carbonated Drinks,2.616741e+06,NaN
